In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import keras
from keras import Input, Model
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import adam
from keras.preprocessing import image
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Activation, Add

Using TensorFlow backend.


In [ ]:
datagen = image.ImageDataGenerator(
   rotation_range=30,
   width_shift_range=0.1,
   height_shift_range=0.1,
   channel_shift_range=40,
   shear_range=0.1,
   zoom_range=0.1,
   horizontal_flip=True,
   fill_mode='nearest')

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

169009152/169001437 [==============================] - 3s 0us/step


In [ ]:
input_shape=(32, 32, 3, )

conv_dropout = 0.20
reg_val = 5e-5

learning_rate = 5e-4
epochs = 4
batch_size = 256
steps = len(x_train) // batch_size
iterations = 100

In [ ]:
def add_conv(x_out, width, kernel_size):
  
  x_out = Conv2D(width, kernel_size=kernel_size, padding='same', kernel_regularizer=l2(reg_val))(x_out)
  x_out = BatchNormalization()(x_out)

  return x_out

In [26]:
def start_block(x_out, width):
  x_out = add_conv(x_out, width, 3)
  x_out = Activation('relu')(x_out)

  return x_out

In [44]:
def conv_block(x_out, width_values):
  x_resid = x_out

  x_out = add_conv(x_out, width_values[0], 3)
  x_out = Activation('relu')(x_out)

  x_out = add_conv(x_out, width_values[1], 3)
  x_out = Activation('relu')(x_out)

  x_out = add_conv(x_out, width_values[2], 3)

  x_resid = add_conv(x_resid, width_values[2], 1)

  x_out = Add()([x_out, x_resid])
  x_out = Activation('relu')(x_out)

  return x_out

In [45]:
def id_block(x_out, width_values):
  x_resid = x_out

  x_out = add_conv(x_out, width_values[0], 3)
  x_out = Activation('relu')(x_out)

  x_out = add_conv(x_out, width_values[1], 3)
  x_out = Activation('relu')(x_out)

  x_out = add_conv(x_out, width_values[2], 3)

  x_out = Add()([x_out, x_resid])
  x_out = Activation('relu')(x_out)
  x_out = Dropout(conv_dropout)(x_out)

  return x_out

In [46]:
# zero pad input

# kernel_initializer

def resnet():
  x_in = Input(input_shape)
  x_out = x_in

  x_out = start_block(x_out, 64)
  x_out = MaxPooling2D(2,2)(x_out)
  x_out = Dropout(conv_dropout)(x_out)

  x_out = conv_block(x_out, [64, 64, 64])
  x_out = id_block(x_out, [64, 64, 64])
  x_out = MaxPooling2D(2,2)(x_out)
  x_out = Dropout(conv_dropout)(x_out)

  x_out = conv_block(x_out, [128, 128, 128])
  x_out = id_block(x_out, [128, 128, 128])
  x_out = MaxPooling2D(2,2)(x_out)
  x_out = Dropout(conv_dropout)(x_out)


  x_out = conv_block(x_out, [256, 256, 256])
  x_out = id_block(x_out, [256, 256, 256])
  x_out = AveragePooling2D(2,2)(x_out)
  x_out = Dropout(conv_dropout)(x_out)

  x_out = Flatten()(x_out)

  x_out = Dense(256, activation='relu')(x_out)
  x_out = Dropout(0.5)(x_out)
  x_out = Dense(100, activation='softmax')(x_out)

  model = Model(inputs = x_in, outputs = x_out)

  return model

In [47]:
model = resnet()
model.compile(optimizer=adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_195 (Conv2D)             (None, 32, 32, 64)   1792        input_10[0][0]                   
__________________________________________________________________________________________________
batch_normalization_195 (BatchN (None, 32, 32, 64)   256         conv2d_195[0][0]                 
__________________________________________________________________________________________________
activation_134 (Activation)     (None, 32, 32, 64)   0           batch_normalization_195[0][0]    
___________________________________________________________________________________________

In [48]:
for i in range(iterations):
  print("\nCycle:",i,"\n")
  model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, verbose=1)
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=steps , validation_data=(x_test, y_test), epochs=epochs)


Cycle: 0 

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 75s 2ms/step - loss: 4.7264 - acc: 0.0191 - val_loss: 4.6057 - val_acc: 0.0316
Epoch 1/4
195/195 [==============================] - 74s 380ms/step - loss: 4.5440 - acc: 0.0358 - val_loss: 4.3604 - val_acc: 0.0504
Epoch 2/4
195/195 [==============================] - 74s 379ms/step - loss: 4.3199 - acc: 0.0542 - val_loss: 4.2346 - val_acc: 0.0661
Epoch 3/4
195/195 [==============================] - 73s 376ms/step - loss: 4.1292 - acc: 0.0824 - val_loss: 3.9948 - val_acc: 0.0926
Epoch 4/4
195/195 [==============================] - 73s 374ms/step - loss: 3.9679 - acc: 0.1039 - val_loss: 3.9395 - val_acc: 0.1133

Cycle: 1 

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 68s 1ms/step - loss: 3.6919 - acc: 0.1474 - val_loss: 3.8827 - val_acc: 0.1209
Epoch 1/4
195/195 [==============================] - 73s 375ms/step 

KeyboardInterrupt: ignored